In [1]:
using LowLevelFEM
import LowLevelFEM as FEM
gmsh.initialize()

[ Info: Precompiling LowLevelFEM [6171b9fb-adbf-4751-adb9-5faded75de07] (cache misses: include_dependency fsize change (2), wrong dep version loaded (2), incompatible header (14))

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up


In [2]:
gmsh.open("body1-2D.geo")

Info    : Reading 'body1-2D.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line)
Info    : Done meshing 1D (Wall 0.000396787s, CPU 0.000407s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00718986s, CPU 0.007187s)
Info    : 98 nodes 198 elements
Info    : Done reading 'body1-2D.geo'


In [3]:
mat = FEM.material("body")
problem = FEM.Problem([mat], type=:PlaneHeatConduction)

Problem("body1-2D", :PlaneHeatConduction, 2, 1, Material[Material("body", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 98, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [4]:
sT = ScalarField(problem, "left", (x, y, z, t) -> 0, steps=100)
sq = ScalarField(problem, "right", (x, y, z, t) -> 100, steps=100)

ScalarField
[[100.0 100.0 … 100.0 100.0; 100.0 100.0 … 100.0 100.0], [100.0 100.0 … 100.0 100.0; 100.0 100.0 … 100.0 100.0], [100.0 100.0 … 100.0 100.0; 100.0 100.0 … 100.0 100.0], [100.0 100.0 … 100.0 100.0; 100.0 100.0 … 100.0 100.0], [100.0 100.0 … 100.0 100.0; 100.0 100.0 … 100.0 100.0], [100.0 100.0 … 100.0 100.0; 100.0 100.0 … 100.0 100.0], [100.0 100.0 … 100.0 100.0; 100.0 100.0 … 100.0 100.0], [100.0 100.0 … 100.0 100.0; 100.0 100.0 … 100.0 100.0]]

In [5]:
supp = FEM.temperatureConstraint("left", T=0)
load = FEM.heatFlux("right", qn=100)

BoundaryCondition("right", nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, 100, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [6]:
#T0 = FEM.initialTemperature(problem, "body", T=0)
T0 = scalarField(problem, "body", (x, y, z) -> 0)

ScalarField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [7]:
#FEM.initialTemperature!("left", T0, T=0)
#T0

In [8]:
K = FEM.heatConductionMatrix(problem)
C = FEM.heatCapacityMatrix(problem)
q = FEM.heatFluxVector(problem, [load])

ScalarField
[0.0; 6.249999999999992; … ; 0.0; 0.0;;]

In [9]:
#FEM.applyBoundaryConditions!(K, C, q, [supp])

In [10]:
λmax = FEM.largestEigenValue(K, C, [supp])

28634.16628528278

In [11]:
ϑ = 0.66
T = FEM.FDM(K, C, q, [supp], T0, 100, 0.0001 * 2 / (abs(1 - 2ϑ) * λmax), ϑ=ϑ)

ScalarField
[0.0 0.0 … 0.0 0.0; 0.0 1.9802689762410587e-5 … 0.0019242385683192382 0.0019437046350406829; … ; 0.0 -9.643040295498215e-10 … -9.197460946360756e-8 -9.288127933796579e-8; 0.0 4.620157204647345e-7 … 4.653577394800196e-5 4.7022829607872353e-5]

In [12]:
ϑ = 0.33
T2 = FEM.FDM(K, C, q, [supp], T0, 100, 0.0001 * 2 / (abs(1 - 2ϑ) * λmax), ϑ=ϑ)

ScalarField
[0.0 0.0 … 0.0 0.0; 0.0 1.8638975666325922e-5 … 0.0018120564012237901 0.0018303968062018006; … ; 0.0 -9.07716404153256e-10 … -8.675496428824317e-8 -8.761264033155266e-8; 0.0 4.3474519768726024e-7 … 4.372567032392609e-5 4.418276610661865e-5]

In [13]:
FEM.showDoFResults(T, :T, visible=true)
FEM.showDoFResults(T2)

2

In [14]:
q1 = FEM.solveHeatFlux(T)

VectorField
[[0.0 0.0024703487566296533 … 0.23725730856305077 0.23962822930334507; 0.0 -0.00029006028168605645 … -0.02786115781760792 -0.028139579523782928; … ; 0.0 0.0024703487566296533 … 0.23725730856305077 0.23962822930334507; 0.0 -0.00029006028168605645 … -0.02786115781760792 -0.028139579523782928], [0.0 0.002693347580688575 … 0.25779373494147984 0.26036083950070127; 0.0 -0.000629713153546245 … -0.059140498534381074 -0.05971773077371608; … ; 0.0 0.002693347580688575 … 0.25779373494147984 0.26036083950070127; 0.0 -0.000629713153546245 … -0.059140498534381074 -0.05971773077371608], [0.0 0.0023047734799704882 … 0.2214327638480918 0.22364644498691819; 0.0 -0.00023991037997634593 … -0.023631014059626722 -0.023873356179531947; … ; 0.0 0.0023047734799704882 … 0.2214327638480918 0.22364644498691819; 0.0 -0.00023991037997634593 … -0.023631014059626722 -0.023873356179531947], [0.0 5.306735204916849e-7 … 4.864773112183023e-5 4.9109200630588735e-5; 0.0 5.616383549283447e-7 … 4.950382795261647e

In [15]:
FEM.showHeatFluxResults(q1, :qvec)

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


3

In [16]:
q2 = FEM.solveHeatFlux(T, DoFResults=true)

VectorField
[0.0 2.665477971801362e-8 … 1.9979073626166724e-6 2.0123018649727294e-6; 0.0 5.462777806139624e-8 … 5.011587875424382e-6 5.059128518839206e-6; … ; 0.0 0.00031037833052086 … 0.030842141893846958 0.03116110905498375; 0.0 -9.009790923534854e-5 … -0.008416189580899132 -0.008497881471385346]

In [17]:
FEM.showDoFResults(q2, :qvec)

4

In [18]:
q3 = FEM.elementsToNodes(q1)

VectorField
[0.0 2.665477971801362e-8 … 1.9979073626166724e-6 2.0123018649727294e-6; 0.0 5.462777806139624e-8 … 5.011587875424382e-6 5.059128518839206e-6; … ; 0.0 0.00031037833052086 … 0.030842141893846958 0.03116110905498375; 0.0 -9.009790923534854e-5 … -0.008416189580899132 -0.008497881471385346]

In [19]:
FEM.showDoFResults(q3, :qvec)

5

In [20]:
err1 = FEM.fieldError(q1)

VectorField
[0.0 5.462777806139633e-8 … 5.011587875424391e-6 5.0591285188392156e-6; 0.0 5.462777806139624e-8 … 5.011587875424382e-6 5.059128518839206e-6; … ; 0.0 0.001423067544448721 … 0.13652814626805665 0.13789109455027027; 0.0 0.0014665824871435652 … 0.13860277548903602 0.13996481239070271]

In [21]:
FEM.showDoFResults(err1, :vector)

6

In [22]:
FEM.openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0
Fontconfig warning: using without calling FcInit()


In [23]:
gmsh.finalize()